<a href="https://colab.research.google.com/github/JamieMcQuire23/ECG-Heartbeat-Categorisation-Dataset/blob/master/Deep_Transferable_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Transferable Representation 

* This notebook is going to provide a PyTorch implementation of the deep transferable network from https://arxiv.org/pdf/1805.00794.pdf

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [35]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [0]:
#read in the mit dataset
mit_train = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_train.csv",header=None)
mit_test = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_test.csv",header=None)

In [0]:
#convert to the correct form

train_vals = mit_train.values
X_train = train_vals[:, :-1]
Y_train = train_vals[:, -1].astype(int)

test_vals = mit_test.values
X_test = test_vals[:,:-1]
Y_test = test_vals[:,:-1].astype(int)


In [0]:
#create the training and validation set from the training data set
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=seed)

# Create the model

In [0]:
input_shape = (1,187)
num_classes = 5
batch_size = 100
learning_rate = 1e-3
n_epochs = 10

In [0]:
class DTR_Block(nn.Module):

  def __init__(self,in_channels,out_channels):

    super(DTR_Block,self).__init__()

    self.conv1 = nn.Conv1d(in_channels=32,out_channels=32,kernel_size=5,padding=2)
    self.relu1 = nn.ReLU()
    self.conv2 = nn.Conv1d(in_channels=32,out_channels=32,kernel_size=5,padding=2)
    self.relu2 = nn.ReLU()
    self.pool = nn.MaxPool1d(kernel_size=5,padding=2)

  def forward(self,x):
    residual = x
    out = self.conv1(x)
    out = self.relu1(out)
    out = self.conv2(out)
    out += residual
    out = self.relu2(out)
    out = self.pool(out)
    return out

In [0]:
class DTR(nn.Module):

  def __init__(self,block,batch_size,input_dim,num_classes=5):
    super(DTR, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=1,out_channels=32,kernel_size=5,padding=2)
    self.layer1 = self.make_layer(block,input_dim,batch_size)
    self.layer2 = self.make_layer(block,input_dim,batch_size)
    self.layer3 = self.make_layer(block,input_dim,batch_size)
    self.layer4 = self.make_layer(block,input_dim,batch_size)
    self.layer5 = self.make_layer(block,input_dim,batch_size)
    self.fc1 = nn.Linear(32,64)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(64,num_classes)

  def make_layer(self,block,input_dim,batch_size):
    layers = []
    layers.append(block(in_channels=32,out_channels=32))
    return nn.Sequential(*layers)     

  def forward(self,x):
    out = self.conv1(x)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    return out

In [92]:
dtr_model = DTR(DTR_Block,batch_size=batch_size,input_dim=187)
dtr_model.cuda()

DTR(
  (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (layer1): Sequential(
    (0): DTR_Block(
      (conv1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (relu1): ReLU()
      (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (relu2): ReLU()
      (pool): MaxPool1d(kernel_size=5, stride=5, padding=2, dilation=1, ceil_mode=False)
    )
  )
  (layer2): Sequential(
    (0): DTR_Block(
      (conv1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (relu1): ReLU()
      (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (relu2): ReLU()
      (pool): MaxPool1d(kernel_size=5, stride=5, padding=2, dilation=1, ceil_mode=False)
    )
  )
  (layer3): Sequential(
    (0): DTR_Block(
      (conv1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (relu1): ReLU()
      (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (relu2): ReLU()
      (pool):

In [0]:
mit_train = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_train.csv",header=None)
mit_test = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_test.csv",header=None)

mit_train, mit_val = train_test_split(mit_train,test_size=0.2,random_state=seed)


train_tensor = torch.Tensor(mit_train.values)
val_tensor = torch.Tensor(mit_val.values)
test_tensor = torch.Tensor(mit_test.values)

trainloader = torch.utils.data.DataLoader(train_tensor,batch_size=batch_size,
                                          shuffle=False,num_workers=2)

validationloader = torch.utils.data.DataLoader(val_tensor,batch_size=batch_size,
                                               shuffle=False,num_workers=2)

testloader = torch.utils.data.DataLoader(test_tensor,batch_size=batch_size,
                                          shuffle=False,num_workers=2)

In [0]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [0]:
import time
from torch.autograd import Variable

use_cuda = True

def trainNet(net, batch_size, n_epochs, learning_rate, trainloader, validationloader):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    n_batches = len(trainloader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(trainloader):
            
            inputs = data[:, :-1]
            labels = data[:, -1].long()
            sizes = inputs.shape[0]
            #inputs form batch_size, features, time_steps
            inputs = inputs.reshape(sizes,1,187)

            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)

            if use_cuda and torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.item()
            total_train_loss += loss_size.item()
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        total = 0
        correct = 0
        for data in validationloader:

            inputs = data[:, :-1]
            labels = data[:, -1].long()
            sizes = inputs.shape[0]
            inputs = inputs.reshape(sizes,1,187)

            if use_cuda and torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.item()

            _, predicted = torch.max(val_outputs.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(validationloader)))

        print("Accuracy of the network on the validation data: %d %%" % (100 * correct/total))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [95]:
trainNet(dtr_model, batch_size, n_epochs, learning_rate, trainloader, validationloader)

===== HYPERPARAMETERS =====
batch_size= 100
epochs= 10
learning_rate= 0.001
Epoch 1, 10% 	 train_loss: 0.10 took: 1.75s
Epoch 1, 20% 	 train_loss: 0.10 took: 1.67s
Epoch 1, 30% 	 train_loss: 0.09 took: 1.69s
Epoch 1, 40% 	 train_loss: 0.10 took: 1.66s
Epoch 1, 50% 	 train_loss: 0.08 took: 1.67s
Epoch 1, 60% 	 train_loss: 0.09 took: 1.67s
Epoch 1, 70% 	 train_loss: 0.10 took: 1.68s
Epoch 1, 81% 	 train_loss: 0.09 took: 1.67s
Epoch 1, 91% 	 train_loss: 0.11 took: 1.66s
Validation loss = 0.09
Accuracy of the network on the validation data: 97 %
Epoch 2, 10% 	 train_loss: 0.07 took: 1.74s
Epoch 2, 20% 	 train_loss: 0.08 took: 1.66s
Epoch 2, 30% 	 train_loss: 0.07 took: 1.68s
Epoch 2, 40% 	 train_loss: 0.08 took: 1.67s
Epoch 2, 50% 	 train_loss: 0.07 took: 1.66s
Epoch 2, 60% 	 train_loss: 0.08 took: 1.67s
Epoch 2, 70% 	 train_loss: 0.08 took: 1.65s
Epoch 2, 81% 	 train_loss: 0.07 took: 1.66s
Epoch 2, 91% 	 train_loss: 0.09 took: 1.66s
Validation loss = 0.08
Accuracy of the network on the va

In [96]:
correct = 0
total = 0

for data in testloader:
  inputs = data[:, :-1]
  labels = data[:, -1].long()
  sizes = inputs.shape[0]
  inputs = inputs.reshape(sizes,1,187)

  if use_cuda and torch.cuda.is_available():
    inputs = inputs.cuda()
    labels = labels.cuda()

  outputs = dtr_model(inputs)
  _, predicted = torch.max(outputs.data,1)
  total += labels.size(0)
  correct += (predicted == labels).sum()

print("Accuracy of the network on the test data: %d %%" % (100 * correct/total))

Accuracy of the network on the test data: 98 %
